In [ ]:
import codecs
import json
from os.path import join
import pickle
import re
from gensim.models import FastText
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
file = open('/home/cyj/公开数据/all_text.txt',"r") 
list001=[]
for line in file:
    line = line.strip('\n')
    list001.append(line) # do somethingfile.close()
file.close()
list002=[]
for i in range(len(list001)):
    l=list001[i].split(' ')
    for j in range(len(l)):
        list002.append(l[j])
from collections import Counter
b = Counter(list002)
u=list(b.keys())
t=list(b.values())

In [ ]:
ty1=len(list002)
save_model_name = "/home/cyj/公开数据/fasttextmodel/FASTTEXT031401.model"
model_w = FastText.load(save_model_name)

In [ ]:
################# Load and Save Data ################

def load_json(rfdir, rfname):
    with codecs.open(join(rfdir, rfname), 'r', encoding='utf-8') as rf:
        return json.load(rf)


def dump_json(obj, wfpath, wfname, indent=None):
    with codecs.open(join(wfpath, wfname), 'w', encoding='utf-8') as wf:
        json.dump(obj, wf, ensure_ascii=False, indent=indent)



def dump_data(obj, wfpath, wfname):
    with open(os.path.join(wfpath, wfname), 'wb') as wf:
        pickle.dump(obj, wf)


def load_data(rfpath, rfname):
    with open(os.path.join(rfpath, rfname), 'rb') as rf:
        return pickle.load(rf)
    
################# Compare Lists ################

def tanimoto(p,q):
    c = [v for v in p if v in q]
    return float(len(c) / (len(p) + len(q) - len(c)))



################# Paper similarity ################

def generate_pair(pubs,outlier): ##求匹配相似度
    dirpath = '/home/cyj/hin+A+O/f1'
    
    paper_org = {}
    paper_conf = {}
    paper_author = {}
    paper_word = {}
    
    temp=set()
    with open(dirpath + "/paper_org.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_org:
                paper_org[p] = []
            paper_org[p].append(a)
    temp.clear()
    
    with open(dirpath + "/paper_conf.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_conf:
                paper_conf[p]=[]
            paper_conf[p]=a
    temp.clear()
    
    with open(dirpath + "/paper_author.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_author:
                paper_author[p] = []
            paper_author[p].append(a)
    temp.clear()
       
    with open(dirpath + "/paper_word.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_word:
                paper_word[p] = []
            paper_word[p].append(a)
    temp.clear()
    
    
    paper_paper = np.zeros((len(pubs),len(pubs)))
    for i,pid in enumerate(pubs):
        if i not in outlier:
            continue
        for j,pjd in enumerate(pubs):
            if j==i:
                continue
            ca=0
            cv=0
            co=0
            ct=0
          
            if pid in paper_author and pjd in paper_author:
                ca = len(set(paper_author[pid])&set(paper_author[pjd]))*1.5
            if pid in paper_conf and pjd in paper_conf and 'null' not in paper_conf[pid]:
                cv = tanimoto(set(paper_conf[pid]),set(paper_conf[pjd]))
            if pid in paper_org and pjd in paper_org:
                co = tanimoto(set(paper_org[pid]),set(paper_org[pjd]))
            if pid in paper_word and pjd in paper_word:
                ct = len(set(paper_word[pid])&set(paper_word[pjd]))/3
                    
            paper_paper[i][j] =ca+cv+co+ct
            
    return paper_paper

    
        
################# Evaluate ################
        
def pairwise_evaluate(correct_labels,pred_labels):
    TP = 0.0  # Pairs Correctly Predicted To SameAuthor
    TP_FP = 0.0  # Total Pairs Predicted To SameAuthor
    TP_FN = 0.0  # Total Pairs To SameAuthor
    list1=[]
    list2=[]
    list3=[]
    for i in range(len(correct_labels)):
        for j in range(i + 1, len(correct_labels)):
            if correct_labels[i] == correct_labels[j]:
                TP_FN += 1
            if pred_labels[i] == pred_labels[j]:
                TP_FP += 1
            if (correct_labels[i] == correct_labels[j]) and (pred_labels[i] == pred_labels[j]):
                TP += 1
    list1.append(TP)
    list2.append(TP_FP)
    list3.append(TP_FN)

    if TP == 0:
        pairwise_precision = 0
        pairwise_recall = 0
        pairwise_f1 = 0
    else:
        pairwise_precision = TP / TP_FP
        pairwise_recall = TP / TP_FN
        pairwise_f1 = (2 * pairwise_precision * pairwise_recall) / (pairwise_precision + pairwise_recall)
    return pairwise_precision, pairwise_recall, pairwise_f1, list1, list2, list3


################# Save Paper Features ################

def save_relation(name_pubs_raw, name): # 保存论文的各种feature
    name_pubs_raw = load_json('/home/cyj/hin+A+O/result', name_pubs_raw)
    ## trained by all text in the datasets. Training code is in the cells of "train word2vec"
    #save_model_name = "/home/cyj/公开数据/word2vecmodel/Aword2vec.model"
    #model_w = word2vec.Word2Vec.load(save_model_name)
    save_model_name = "/home/cyj/公开数据/fasttextmodel/FASTTEXT031401.model"
    model_w = FastText.load(save_model_name)
    
    r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
    stopword = ['at','based','in','of','for','on','and','to','an','using','with','the','by','we','be','is','are','can']
    stopword1 = ['university','univ','china','department','dept','laboratory','lab','school','al','et',
                 'institute','inst','college','chinese','beijing','journal','science','international']
    
    f1 = open ('/home/cyj/hin+A+O/f1/paper_author.txt','w',encoding = 'utf-8')
    f2 = open ('/home/cyj/hin+A+O/f1/paper_conf.txt','w',encoding = 'utf-8')
    f3 = open ('/home/cyj/hin+A+O/f1/paper_word.txt','w',encoding = 'utf-8')
    f4 = open ('/home/cyj/hin+A+O/f1/paper_org.txt','w',encoding = 'utf-8')
    f5 = open ('/home/cyj/公开数据/database/英文停用词表.txt','r')
    strchword = f5.read()     #将txt文件的所有内容读入到字符串str1中
    f5.close()
    
    taken = name.split("_")
    name = taken[0] + taken[1]
    name_reverse = taken[1]  + taken[0]
    if len(taken)>2:
        name = taken[0] + taken[1] + taken[2]
        name_reverse = taken[2]  + taken[0] + taken[1]
    
    authorname_dict={}
    ptext_emb = {}  
    
    tcp=set()  
    for i,pid in enumerate(name_pubs_raw):
        
        pub = name_pubs_raw[pid]
        
        #save authors
        org=""
        for author in pub["authors"]:
            authorname = re.sub(r,'', author["name"]).lower()
            taken = authorname.split(" ")
            if len(taken)==2: ##检测目前作者名是否在作者词典中
                authorname = taken[0] + taken[1]
                authorname_reverse = taken[1]  + taken[0] 
            
                if authorname not in authorname_dict:
                    if authorname_reverse not in authorname_dict:
                        authorname_dict[authorname]=1
                    else:
                        authorname = authorname_reverse 
            else:
                authorname = authorname.replace(" ","")
            
            if authorname!=name and authorname!=name_reverse:
                f1.write(pid + '\t' + authorname + '\n')
        
            else:
                if "org" in author:
                    org = author["org"]
                    
                    
        #save org 待消歧作者的机构名
        pstr = org.strip()
        pstr = pstr.lower() #小写
        pstr = re.sub(r,' ', pstr) #去除符号
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip() #去除多余空格
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in strchword]
        pstr=set(pstr)
        for word in pstr:
            f4.write(pid + '\t' + word + '\n')

        
        #save venue
        pstr = pub["venue"].strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in strchword]
        for word in pstr:
            f2.write(pid + '\t' + word + '\n')
        if len(pstr)==0:
            f2.write(pid + '\t' + 'null' + '\n')

            
        #save text
        pstr = ""    
        keyword=""
        if "keywords" in pub:
            for word in pub["keywords"]:
                keyword=keyword+word+" "
        pstr = pstr + pub["title"]
        pstr=pstr.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in strchword]
        for word in pstr:
            f3.write(pid + '\t' + word + '\n')
        
        #save all words' embedding
        pstr = keyword + " " + pub["title"] + " " + pub["venue"] + " " + org
        if "year" in pub:
              pstr = pstr +  " " + str(pub["year"])
        pstr=pstr.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>2]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in strchword]

        words_vec=[]
        for word in pstr:
            if ((word in model_w) and (word in u)):
                ty2=u.index(word)
                ty3=t[ty2]
                m1=1-(ty3/ty1)
                words_vec.append(model_w[word]*m1)
        if len(words_vec)<1:
            words_vec.append(np.zeros(100))
            tcp.add(i)
            #print ('outlier:',pid,pstr)
        ptext_emb[pid] = np.mean(words_vec,0)
        
    #  ptext_emb: key is paper id, and the value is the paper's text embedding
    dump_data(ptext_emb,'/home/cyj/hin+A+O/f1','ptext_emb.pkl')
    # the paper index that lack text information
    dump_data(tcp,'/home/cyj/hin+A+O/f1','tcp.pkl')
            
 
    f1.close()
    f2.close()
    f3.close()
    f4.close()

In [ ]:
import pandas as pd
def save_generatenode(name_pubs_raw, name): # 保存论文的各种feature
    pubs_raw = load_json('/home/cyj/hin+A+O/result', name_pubs_raw)
    #pubs_raw = load_json("/home/cyj/数据/database","li_guo.json")
    pubs_raw1=list(pubs_raw.keys())
    pubs_raw2=list(pubs_raw.values())
    list01=[]
    list02=[]
    list03=[]
    list04=[]
    for i in range(len(pubs_raw2)):
        a = pubs_raw2[i]['authors']
        for j in range(len(a)):
            list01.append(a[j]['name'])
            t=list(a[j].keys())
            if len(t) == 1:
                list02.append('')
            #elif t[1] == 'org':
            else:
                list02.append(a[j]['org'])
        list03.append(list01)
        list04.append(list02)
        list01=[]
        list02=[]
    org1=[]
    org2=[]
    for i in range(len(list04)):
        for j in range(len(list04[i])):
            #if list04[i][j] != '':
                org1.append(list04[i][j])
        org2.append(org1)
        org1=[]
    org3=[]
    for i in range(len(org2)):
        if org2[i] != []:
            org3.append(org2[i])
    aut=[]
    aut1=[]
    for i in range(len(list03)):
        for j in range(len(list03[i])):
            rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
            line = rule.sub('',list03[i][j]).lower()
            aut1.append(line)
        aut.append(aut1)
        aut1=[]
    org4 = []
    org5 = []
    for i in range(len(aut)):
        for j in range(len(aut[i])):
            name1=name.replace('_','').lower()
            if aut[i][j] != name1:
                org4.append(org3[i][j])
        org5.append(org4)
        org4 = []
    org6=[]
    org7=[]
    for i in range(len(org5)):
        for j in range(len(org5[i])):
            if org5[i][j] != '':
                org6.append(org5[i][j])
        org7.append(org6)
        org6=[]
    org9=[]
    org10=[]
    for i in range(len(org7)):
        if org7[i] != []:
            org9.append(org7[i])
            org10.append(pubs_raw1[i])
    r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
    stopword = ['at','based','in','of','for','on','and','to','an','using','with','the','by','we','be','is','are','can']
    stopword1 = ['university','univ','china','department','dept','laboratory','lab','school','al','et',
                 'institute','inst','college','chinese','beijing','journal','science','international']
    f5 = open ('/home/cyj/公开数据/database/英文停用词表.txt','r')
    strchword = f5.read()     #将txt文件的所有内容读入到字符串str1中
    f5.close()
    org11=[]
    for i in range(len(org9)):
        a=''.join(org9[i])
        pstr = a.strip()
        pstr = pstr.lower() #小写
        pstr = re.sub(r,' ', pstr) #去除符号
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip() #去除多余空格
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in strchword]
        org11.append(pstr)
    org12=[]
    org13=[]
    org14=[]
    org15=[]
    org16=[]
    org17=[]
    for i in range(len(org11)):
        #print(i)
        l=len(org11)
        u=org11[i+1:l]
        g=org10[i+1:l]
        for j in range(len(u)):
            x = [k for k in org11[i] if k in u[j]]
            if len(x) != 0:
                org12.append(pubs_raw1[i])
                org13.append(g[j])
                org14.append(len(x))
                org15.append('Ti')
                org16.append('Ti')
                org17.append('Oi-Oi')
    t1=[]
    t2=[]
    L=[]
    t3=[]
    t4=[]
    t5=[]
    for i in range(len(aut)):
        l=len(aut)
        u=aut[i+1:l]
        g=pubs_raw1[i+1:l]
        for j in range(len(u)):
            x = [k for k in aut[i] if k in u[j]]
            if len(x) != 1:
                t1.append(pubs_raw1[i])
                t2.append(g[j])
                L.append(len(x)-1)
                t3.append('Ti')
                t4.append('Ti')
                t5.append('Ai-Ai')
    n1=org12+t1
    n2=org13+t2
    n3=org14+L
    n4=org15+t3
    n5=org16+t4
    n6=org17+t5
    array1 = np.array(n1)
    df09=pd.DataFrame(array1)
    df09.columns=["dest_node"]
    array2 = np.array(n2)
    df010=pd.DataFrame(array2)
    df010.columns=["source_node"]
    array3 = np.array(n3)
    df011=pd.DataFrame(array3)
    df011.columns=["weight"]
    array4 = np.array(n4)
    df012=pd.DataFrame(array4)
    df012.columns=["source_class"]
    array5 = np.array(n5)
    df013=pd.DataFrame(array5)
    df013.columns=["dest_class"]
    array6 = np.array(n6)
    df014=pd.DataFrame(array6)
    df014.columns=["edge_class"]
    df016=pd.concat([df09,df010,df011,df012,df013,df014],axis=1)
    ll=t1+t2+org12+org13
    uu=list(set(ll))
    list007=[]
    for i in range(len(pubs_raw1)):
        if pubs_raw1[i] not in uu:
            list007.append(pubs_raw1[i])
    return df016,list007

In [ ]:
import random

import pandas as pd
import networkx as nx
from itertools import product
from collections import defaultdict

class HIN:
    """
    Class to generate vertex sequences.
    """

    def __init__(self, window=None):
        self.graph = nx.DiGraph()
        self.node_size = 0
        self._path_size = 0

        def new_id():
            i = self.node_size
            self.node_size += 1
            return i

        self._node2id = defaultdict(new_id)
        self._id2type = {}
        self._window = window
        self._node_types = set()
        self._path2id = None
        self._id2path = None
        self._id2node = None

    @property
    def id2node(self):
        return self._id2node

    @property
    def id2path(self):
        return self._id2path

    @property
    def window(self):
        return self._window

    @window.setter
    def window(self, val):
        if not self._window:
            self._window = val
        else:
            raise ValueError("window只能被设定一次")

    @property
    def path_size(self):
        if not self._path_size:
            raise ValueError("run sample() first to count path size")
        return self._path_size

    def add_edge(self, source_node, source_class, dest_node, dest_class, edge_class, weight):
        i = self._node2id[source_node]
        j = self._node2id[dest_node]
        self._id2type[i] = source_class
        self._id2type[j] = dest_class
        self._node_types.add(source_class)
        self._node_types.add(dest_class)
        self.graph.add_edge(i, j, weight=weight)

    def small_walk(self, start_node, length):
        walk = [start_node]
        for i in range(1, length):
            if next(nx.neighbors(self.graph, walk[-1]), None) is None:
                break
            cur_node = walk[-1]
            nodes = list(nx.neighbors(self.graph, cur_node))
            weights = [self.graph[cur_node][i]['weight'] for i in nodes]  # 有向图可能不能这么做
            s = sum(weights)
            if s != 0:
                weights = [i/s for i in weights]
                #walk += random.choices(nodes, weights, k=1)
                walk += random.sample(list(nx.neighbors(self.graph, cur_node)), 1)  # todo 添加按权重游走的采样方式
        return walk

    def do_walks(self, length):
        for start_node in range(self.node_size):
            yield self.small_walk(start_node, length)

    def sample(self, length, n_repeat):
        """
        从随机游走的结果中截取结果返回，每个节点轮流作为起始节点
        :param length: 游走长度
        :param n_repeat: 游走次数
        :return: （start_id,end_id,edge_type)
        """
        if not self.window:
            raise ValueError("window not set")

        if not self._path2id:
            self._path2id = {}
            path_id = 0
            for w in range(1, self._window + 1):
                for i in product(self._node_types, repeat=w + 1):
                    self._path2id[i] = path_id
                    path_id += 1

            self._path_size = len(self._path2id)
            self._id2node = {v: k for k, v in self._node2id.items()}
            self._id2path = {v: k for k, v in self._path2id.items()}

        samples = []

        for repeat in range(n_repeat):
            for walk in self.do_walks(length):
                cur_len = 0
                for i, node_id in enumerate(walk):
                    cur_len = min(cur_len + 1, self._window + 1)  # 当window=n的时候，最长路径有n+1个节点
                    if cur_len >= 2:
                        for path_length in range(1, cur_len):
                            sample = (walk[i - path_length], walk[i],
                                      self._path2id[tuple([self._id2type[t] for t in walk[i - path_length:i + 1]])])
                            # print(tuple([self.id2type[t] for t in walk[i-path_length:i + 1]]))
                            samples.append(sample)

        return samples

    def print_statistics(self):
        print(f'size = {self.node_size}')
        
######################################构建图#######################3    
def load_a_HIN_from_pandas(edges, print_graph=False):
    """
    单向边：edges = list(pd.df)
    """

    def reverse(df):
        """
        reverse source & dest
        """
        df = df.rename({'source_node': 'dest_node', 'dest_node': 'source_node',
                        'source_class': 'dest_class', 'dest_class': 'source_class'},
                       axis=1)
        # reverse edge_class
        df.edge_class = df.edge_class.map(lambda x: '-'.join(reversed(x.split('-'))))
        return df

    print('load graph from edges...')
    g = HIN()
    if isinstance(edges, list):
        edges = pd.concat(edges, sort=False)
    edges = edges.append(reverse(edges), sort=False, ignore_index=True)

    for index, row in edges.iterrows():
        g.add_edge(row['source_node'], row['source_class'],
                   row['dest_node'], row['dest_class'], row['edge_class'],
                   row['weight'])
    if print_graph:
        g.print_statistics()
    print('finish loading graph!')
    return g

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class NSTrainSet(Dataset):
    """
    完全随机的负采样 todo 改进一下？
    """

    def __init__(self, sample, node_size, neg=5):
        """
        :param node_size: 节点数目
        :param neg: 负采样数目
        :param sample: HIN.sample()返回值，(start_node, end_node, path_id)
        """

        print('init training dataset...')

        l = len(sample)

        x = np.tile(sample, (neg + 1, 1))
        y = np.zeros(l * (1 + neg))
        y[:l] = 1

        # x[l:, 2] = np.random.randint(0, path_size - 1, (l * neg,))
        x[l:, 1] = np.random.randint(0, node_size - 1, (l * neg,))

        self.x = torch.LongTensor(x)
        self.y = torch.FloatTensor(y)
        self.length = len(x)

        print('finished')

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.length

In [ ]:
class HIN2vec(nn.Module):

    def __init__(self, node_size, path_size, embed_dim, sigmoid_reg=False, r=True):
        super().__init__()

        # self.args = args

        def binary_reg(x: torch.Tensor):
            raise NotImplementedError()
            # return (x >= 0).float()  # do not have gradients

        self.reg = torch.sigmoid if sigmoid_reg else binary_reg

        self.__initialize_model(node_size, path_size, embed_dim, r)

    def __initialize_model(self, node_size, path_size, embed_dim, r):
        self.start_embeds = nn.Embedding(node_size, embed_dim)
        self.end_embeds = self.start_embeds if r else nn.Embedding(node_size, embed_dim)

        self.path_embeds = nn.Embedding(path_size, embed_dim)
        # self.classifier = nn.Sequential(
        #     nn.Linear(embed_dim, 1),
        #     nn.Sigmoid(),
        # )

    def forward(self, start_node: torch.LongTensor, end_node: torch.LongTensor, path: torch.LongTensor):
        # assert start_node.dim() == 1  # shape = (batch_size,)

        s = self.start_embeds(start_node)  # (batch_size, embed_size)
        e = self.end_embeds(end_node)
        p = self.path_embeds(path)
        p = self.reg(p)

        agg = torch.mul(s, e)
        agg = torch.mul(agg, p)
        # agg = F.sigmoid(agg)
        # output = self.classifier(agg)

        output = torch.sigmoid(torch.sum(agg, axis=1))

        return output

In [ ]:
def train(log_interval, model, device, train_loader: DataLoader, optimizer, loss_function, epoch):
    model = model.to(device)
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data[:, 0], data[:, 1], data[:, 2])
        loss = loss_function(output.view(-1), target)
        loss.backward()
        optimizer.step()

        if idx % log_interval == 0:
            print(f'\rTrain Epoch: {epoch} '
                  f'[{idx * len(data)}/{len(train_loader.dataset)} ({100. * idx / len(train_loader):.3f}%)]\t'
                  f'Loss: {loss.item():.3f}\t\t',
                  # f'data = {data}\t target = {target}',
                  end='')
    print()

In [ ]:
pubs_raw = load_json("/home/cyj/公开数据/database","train_pub.json")
name_pubs = load_json("/home/cyj/公开数据/database","train_author.json")
import re
from gensim.models import word2vec
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
import codecs
import json
from os.path import join
import pickle
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


result=[]
result1=[]
result2=[]
count1=[]

listn1=[]
listn2=[]
listn3=[]

for n,name in enumerate(name_pubs):
    ilabel=0
    pubs=[] # all papers
    labels=[] # ground truth
    
    for author in name_pubs[name]:
        iauthor_pubs = name_pubs[name][author]
        for pub in iauthor_pubs:
            pubs.append(pub)
            labels.append(ilabel)
        ilabel += 1
        
    print (n,name,len(pubs))
    #print(pubs)
    #count1.append(len(pubs))
    
    
    if len(pubs)==0:
        result.append(0)
        continue
    
    ##保存关系
    ###############################################################
    name_pubs_raw = {}
    for i,pid in enumerate(pubs):
        name_pubs_raw[pid] = pubs_raw[pid]
        
    dump_json(name_pubs_raw, '/home/cyj/hin+A+O/result', name+'.json', indent=4)
    save_relation(name+'.json', name)  
    ###############################################################
    
    
   
    ''' ##元路径游走类
    ###############################################################r
    mpg = MetaPathGenerator()
    mpg.read_data("/home/cyj/数据/f1")
    ###############################################################'''
    

    
    ##论文关系表征向量
    ############################################################### 
    # 
    tt=save_generatenode(name+'.json', name)
    window = 5
    walk = 5
    walk_length = 20
    embed_size = 100
    neg = 3
    sigmoid_reg = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f'device = {device}')

    # set dataset [PLEASE USE YOUR OWN DATASET TO REPLACE THIS]
    demo_edge = tt[0]

    edges = [demo_edge]

    hin = load_a_HIN_from_pandas(edges)
    hin.window = window
    dataset = NSTrainSet(hin.sample(walk_length, walk), hin.node_size, neg=neg)
    hin2vec = HIN2vec(hin.node_size, hin.path_size, embed_size, sigmoid_reg)

    n_epoch = 5
    batch_size = 20
    log_interval = 200

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(hin2vec.parameters())  # 原作者使用的是SGD？ 这里使用AdamW
    loss_function = nn.BCELoss()
    for epoch in range(n_epoch):
        train(log_interval, hin2vec, device, data_loader, optimizer, loss_function, epoch)
    
    namelist=list(hin.id2node.values())
    node_embeds = pd.DataFrame(hin2vec.start_embeds.weight.data.cpu().numpy()).T

    #all_embs=[]
    embs=[]
    cp = set()
    for i,pid in enumerate(pubs):
        if pid in namelist:
            for j in range(len(namelist)):
                if pid == namelist[j]:
                    embs.append(node_embeds[j].values)
        else:
            cp.add(i)
            embs.append(np.zeros(100))
    #all_embs.append(embs)
    all_embs= np.array(embs)
    print ('relational outlier:',cp)
    ############################################################### 

    
    
    ##论文文本表征向量
    ###############################################################   
    ptext_emb=load_data('/home/cyj/hin+A+O/f1','ptext_emb.pkl')
    tcp=load_data('/home/cyj/hin+A+O/f1','tcp.pkl')
    print ('semantic outlier:',tcp)
    tembs=[]
    for i,pid in enumerate(pubs):
        tembs.append(ptext_emb[pid])
    ############################################################### 
    
    ##离散点
    outlier=set()
    for i in cp:
        outlier.add(i)
    for i in tcp:
        outlier.add(i)
    #print('离散点:',outlier)
    
    ##网络嵌入向量相似度
    #sk_sim = np.zeros((len(pubs),len(pubs)))
    #for k in range(rw_num):
    sk_sim = pairwise_distances(all_embs,metric="cosine")
    #sk_sim =sk_sim/rw_num    
    
    ##文本相似度
    t_sim = pairwise_distances(tembs,metric="cosine")
    
    w=1
    sim = (np.array(sk_sim) + w*np.array(t_sim))/(1+w)
    
    
    
    ##evaluate
    ###############################################################
    pre = DBSCAN(eps = 0.2, min_samples = 4,metric ="precomputed").fit_predict(sim)
    #print(pre)
    #print(type(pre))
    #for i in range(len(pre)):
     #   print('pre',[i],pre[i])
    #print('len:',len(pre))
    
    for i in range(len(pre)):
        if pre[i]==-1:
            outlier.add(i)
    #print('outlier:',outlier)
    ## assign each outlier a label
    paper_pair = generate_pair(pubs,outlier)
    #print('paper_pair:',paper_pair)
    paper_pair1 = paper_pair.copy()
    K = len(set(pre))
    
    listap=[]
    for i in range(len(pubs)):
        listap.append(float(-1))
    na1=np.array(listap)
    
    
    for i in range(len(pre)):
        if i not in outlier:
            continue
        j = np.argmax(paper_pair[i])
        #print('j1:',j)
        while j in outlier:
            paper_pair[i][j]=-1
            if any(na1==paper_pair[i])  == True:
                break
            else:
                j = np.argmax(paper_pair[i])
            #print('j2:',j)
        if paper_pair[i][j]>=1.5:
            pre[i]=pre[j]
        else:
            pre[i]=K
            K=K+1
    
    ## find nodes in outlier is the same label or not
    for ii,i in enumerate(outlier):
        for jj,j in enumerate(outlier):
            if jj<=ii:
                continue
            else:
                if paper_pair1[i][j]>=1.5:
                    pre[j]=pre[i]
            
            
    
    labels = np.array(labels)
    pre = np.array(pre)
    print (labels,len(set(labels)))
    print (pre,len(set(pre)))
    pairwise_precision, pairwise_recall, pairwise_f1, a, b, c = pairwise_evaluate(labels,pre)
    print (pairwise_precision, pairwise_recall, pairwise_f1, a, b, c)
    result.append(pairwise_precision)
    result1.append(pairwise_recall)
    result2.append(pairwise_f1)

    print ('avg_pre:', np.mean(result),'avg_recall:', np.mean(result1),'avg_f1:', np.mean(result2))
    listn1.append(np.mean(result))
    listn2.append(np.mean(result1))
    listn3.append(np.mean(result2))